In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math as ma
warnings.filterwarnings("ignore")

# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

In [4]:
loaded_model = joblib.load("finalized_model.sav")

In [5]:
df = pd.read_csv("New_Customers.csv")
df.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_industry_category,wealth_segment,owns_car,tenure,state,...,Unnamed: 16369,Unnamed: 16370,Unnamed: 16371,Unnamed: 16372,Unnamed: 16373,Unnamed: 16374,Unnamed: 16375,Unnamed: 16376,Unnamed: 16377,Unnamed: 16378
0,Chickie,Brister,Male,86,1957-07-12,Manufacturing,Mass Customer,Yes,14,QLD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Morly,Genery,Male,69,1970-03-22,Property,Mass Customer,No,16,NSW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ardelis,Forrester,Female,10,1974-08-28,Financial Services,Affluent Customer,No,10,VIC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lucine,Stutt,Female,64,1979-01-28,Manufacturing,Affluent Customer,Yes,5,QLD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Services,Affluent Customer,No,19,NSW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.drop(df.columns[13:],axis=1, inplace=True)

In [12]:
df.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_industry_category,wealth_segment,owns_car,tenure,state,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,Manufacturing,Mass Customer,Yes,14,QLD,6.0,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Property,Mass Customer,No,16,NSW,11.0,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Financial Services,Affluent Customer,No,10,VIC,5.0,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Manufacturing,Affluent Customer,Yes,5,QLD,1.0,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Services,Affluent Customer,No,19,NSW,9.0,4,1.703125


In [16]:
df_processed = pd.read_csv("df_processed_final.csv")
df_processed.head()

,Unnamed: 0,customer_id,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,Age,online_order,product_class,list_price,New South Wales,QLD,VIC,Victoria
0,0,1,0,93,0.15,2,1,11.0,10,66.0,11,1,9084.45,1,0,0,0
1,1,2,1,81,0.19,2,1,16.0,10,39.0,3,1,4149.07,1,0,0,0
2,2,4,1,33,0.05,2,0,7.0,9,59.0,2,1,1047.72,0,1,0,0
3,3,5,0,56,NaN,0,1,8.0,4,43.0,6,1,5903.20,1,0,0,0
4,4,6,1,35,0.09,1,1,13.0,9,54.0,5,1,5931.69,0,0,1,0


In [160]:
df_new = df.copy()

In [161]:
#https://stackoverflow.com/questions/26788854/pandas-get-the-age-from-a-date-example-date-of-birth
# Computing age
df_new["DOB"] = pd.to_datetime(df_new["DOB"])
now = pd.Timestamp('now')
df_new['DOB'].where(df_new['DOB'] < now, df_new['DOB'] -  np.timedelta64(100, 'Y'))   # 2
df_new["Age"] = (now - df_new['DOB']).astype('<m8[Y]')   
df_new.head(2)

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_industry_category,wealth_segment,owns_car,tenure,state,property_valuation,Rank,Value,Age
0,Chickie,Brister,Male,86,1957-07-12,Manufacturing,Mass Customer,Yes,14,QLD,6.0,1,1.71875,63.0
1,Morly,Genery,Male,69,1970-03-22,Property,Mass Customer,No,16,NSW,11.0,1,1.71875,50.0


In [162]:
industry_proportion = round(df_new.groupby("job_industry_category")["past_3_years_bike_related_purchases"].sum()/df_new["past_3_years_bike_related_purchases"].sum(),2)
industry_map = {'Argiculture': 0.03,
 'Entertainment': 0.03,
 'Financial Services': 0.19,
 'Health': 0.15,
 'IT': 0.05,
 'Manufacturing': 0.2,
 'Property': 0.07,
 'Retail': 0.09,
 'Telecommunications': 0.02}

In [163]:
# Map job_industry_category column
df_new["job_industry_category"] = df_new["job_industry_category"].map(industry_map)
df_new.job_industry_category[:5]

0    0.20
1    0.07
2    0.19
3    0.20
4    0.19
Name: job_industry_category, dtype: float64

In [164]:
# Encode wealth segment
wealth_map = {'Affluent Customer':0, 'High Net Worth': 1, 'Mass Customer':2}
df_new["wealth_segment"] = df_new["wealth_segment"].map(wealth_map)
df_new.wealth_segment.unique()

array([2, 0, 1], dtype=int64)

In [165]:
# # Encode owns car
car_map = {"Yes": 1, "No": 0}
df_new["owns_car"] = df_new["owns_car"].map(car_map)
df_new["owns_car"].unique()

array([1, 0], dtype=int64)

In [166]:
# OneHotencoding of state
states = pd.get_dummies(df_new["state"], drop_first=False)
states

,NSW,QLD,VIC
0,0,1,0
1,1,0,0
2,0,0,1
3,0,1,0
4,1,0,0
...,...,...,...
995,1,0,0
996,1,0,0
997,0,1,0
998,0,1,0


In [167]:
#states = pd.get_dummies(df_new["state"])#, drop_first=True)
states.rename(columns = {"NSW":"New South Wales"}, inplace=True)
states["Victoria"] = 0

states

,New South Wales,QLD,VIC,Victoria
0,0,1,0,0
1,1,0,0,0
2,0,0,1,0
3,0,1,0,0
4,1,0,0,0
...,...,...,...,...
995,1,0,0,0
996,1,0,0,0
997,0,1,0,0
998,0,1,0,0


In [168]:
df_new = pd.concat([df_new,states], axis=1)
df_new.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_industry_category,wealth_segment,owns_car,tenure,state,property_valuation,Rank,Value,Age,New South Wales,QLD,VIC,Victoria
0,Chickie,Brister,Male,86,1957-07-12,0.20,2,1,14,QLD,6.0,1,1.718750,63.0,0,1,0,0
1,Morly,Genery,Male,69,1970-03-22,0.07,2,0,16,NSW,11.0,1,1.718750,50.0,1,0,0,0
2,Ardelis,Forrester,Female,10,1974-08-28,0.19,0,0,10,VIC,5.0,1,1.718750,46.0,0,0,1,0
3,Lucine,Stutt,Female,64,1979-01-28,0.20,0,1,5,QLD,1.0,4,1.703125,41.0,0,1,0,0
4,Melinda,Hadlee,Female,34,1965-09-21,0.19,0,0,19,NSW,9.0,4,1.703125,55.0,1,0,0,0


In [169]:
# Drop unecessary columns
df_new.drop(["state", "Rank", "Value"], axis=1, inplace=True )
df_new.drop(["first_name", "last_name"], axis=1, inplace=True )
df_new.drop(["DOB"], axis=1, inplace=True )

In [170]:
# Map gender column
m = {"Male":1,"Female":0, "U":0}
df_new["gender"] = df_new["gender"].map(m)
df_new.head(1)

,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,Age,New South Wales,QLD,VIC,Victoria
0,1,86,0.2,2,1,14,6.0,63.0,0,1,0,0


In [171]:
df_new["online_order"] = 0
df_new["product_class"] = 0
df_new.head(2)

,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,Age,New South Wales,QLD,VIC,Victoria,online_order,product_class
0,1,86,0.20,2,1,14,6.0,63.0,0,1,0,0,0,0
1,1,69,0.07,2,0,16,11.0,50.0,1,0,0,0,0,0


In [172]:
df_new = df_new[['gender','past_3_years_bike_related_purchases', 'job_industry_category',
       'wealth_segment', 'owns_car', 'tenure', 'property_valuation', 'Age',
       'online_order', 'product_class', 'New South Wales', 'QLD',
       'VIC', 'Victoria']]

In [173]:
df_new.head()

,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,Age,online_order,product_class,New South Wales,QLD,VIC,Victoria
0,1,86,0.20,2,1,14,6.0,63.0,0,0,0,1,0,0
1,1,69,0.07,2,0,16,11.0,50.0,0,0,1,0,0,0
2,0,10,0.19,0,0,10,5.0,46.0,0,0,0,0,1,0
3,0,64,0.20,0,1,5,1.0,41.0,0,0,0,1,0,0
4,0,34,0.19,0,0,19,9.0,55.0,0,0,1,0,0,0


In [174]:
df_new.isna().sum()

gender                                   0
past_3_years_bike_related_purchases      0
job_industry_category                  165
wealth_segment                           0
owns_car                                 0
tenure                                   0
property_valuation                       0
Age                                     17
online_order                             0
product_class                            0
New South Wales                          0
QLD                                      0
VIC                                      0
Victoria                                 0
dtype: int64

In [175]:
df_new["job_industry_category"].fillna(df_new["job_industry_category"].median(), inplace=True)
df_new["Age"].fillna(df_new["Age"].median(), inplace=True)

In [176]:
df_new.isna().sum().sum()

0

In [177]:
df_new.dtypes

gender                                   int64
past_3_years_bike_related_purchases      int64
job_industry_category                  float64
wealth_segment                           int64
owns_car                                 int64
tenure                                   int64
property_valuation                     float64
Age                                    float64
online_order                             int64
product_class                            int64
New South Wales                          uint8
QLD                                      uint8
VIC                                      uint8
Victoria                                 int64
dtype: object

In [179]:
X = df_new.values

In [180]:
# Standardizing the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_test = scaler.fit_transform(X)

In [206]:
# predict list price
index = []
customers = []
for i in range(1000):
    index.append(i)
    customers.append(loaded_model.predict([X_test[i]])[0])

In [216]:
customers_df = pd.DataFrame(customers, index)
customers_df.columns = ["list_price"]
customers_df.head()


,list_price
0,6294.765093
1,7011.456503
2,6823.118411
3,6518.037128
4,6495.230619


In [221]:
customers_final = customers_df.sort_values(by = "list_price", ascending = False)[:100]


In [243]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(customers_final)

In [244]:
customers_to_target = pd.concat([df1,customers_final], axis=1)
customers_to_target.head()

,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,Age,online_order,product_class,New South Wales,QLD,VIC,Victoria,list_price
443,0,59,0.05,2,0,6,2.0,56.0,0,0,1,0,0,0,7539.391147
715,0,82,0.02,2,1,10,2.0,56.0,0,0,1,0,0,0,7493.464066
684,1,58,0.09,1,0,7,2.0,43.0,0,0,0,0,1,0,7481.172935
376,1,60,0.05,1,0,10,2.0,45.0,0,0,0,1,0,0,7469.146009
212,1,12,0.09,2,0,6,6.0,23.0,0,0,0,0,1,0,7467.765292


In [245]:
customers_to_target.to_csv("customers_to_target.csv")